# Text Summarization of User Reviews with Generative Models on Vertex AI

__Notebook Holder__ : Himanshu Pant and Ishan Bhatt

## Overview
Text summarization produces a concise and fluent summary of a longer text document. There are two main text summarization types: extractive and abstractive. Extractive summarization involves selecting critical sentences from the original text and combining them to form a summary. Abstractive summarization involves generating new sentences representing the original text's main points. In this notebook, you go through a few examples of how large language models can help with generating summaries based on text.

Learn more about text summarization in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/summarization-prompts).

### Objective
To summarise user reviews and give prometheus bot suggestions to buy a product or not.

## Getting Started

### Import libraries

Let's start by importing the libraries that we will need for this tutorial

In [ ]:
!pip install gradio==3.41.0

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [33]:
import gradio as gr
import json
import tqdm as notebook_tqdm
from IPython.display import Markdown
import vertexai
from vertexai.language_models import TextGenerationModel
vertexai.init(project="wmt-7fbls2a91f025anb93e025b02g", location="us-central1")

### Import models

Here we load the pre-trained text generation model called `text-bison@001`.

In [34]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## List of themes available in gradio

In [35]:
Markdown(
"""
* gr.themes.Base()
* gr.themes.Default()
* gr.themes.Glass()
* gr.themes.Monochrome()
* gr.themes.Soft()
""")


* gr.themes.Base()
* gr.themes.Default()
* gr.themes.Glass()
* gr.themes.Monochrome()
* gr.themes.Soft()


### User Review Summary:

In [110]:
def generate_summary_few_shots(input_url):
    generate_from_web = {}
    emoji = "🟡"
    parameters = {
    "max_output_tokens": 1024,
    "temperature": 0.2,
    "top_p": 0.95,
    "top_k": 40
    }
    response = generation_model.predict(
        """You are an Review Summarisation Tool, which provides concise summary of users reviews of products purchased from Walmart.com. Analyzing Customer Sentiment and Reviews for Walmart.com.

Focus on extracting key sentiments, prevalent themes and overall satisfaction levels conveyed in the reviews as well as sentiments related to specific categories: price, shipping, delivery, quality, customer service, and features/functionality. Provide both positive and negative feedback where applicable, capturing the general consensus of customer\'s experience.

The coherent summary output should consider below properties wherever applicable:
1. Exclude offensive words from the output.
2. Include overall Sentiment based on these reviews
3. Include Sentiment towards the price point
4. Include Sentiment towards shipping efficiency
5. Include Sentiment towards delivery experience
6. Include Sentiment towards product quality
7. Include Sentiment towards customer service experience
8. Include Sentiment towards product features and functionality
9. Include recommendations to buy the product or not.

Also, classify the coherent summary from the following 3 categories:
    1. Positive
    2. Negative
    3. Neutral

Respond only in consistent json format with summary under \"summary\" key and class under the \"class\" key.

input: Craft a coherent summary in maximum of 100 words of user reviews for Walmart.com that can be parsed from: https://www.walmart.com/ip/Free-Assembly-Women-s-Patch-Pocket-Cardigan-Sweater-Midweight-Sizes-XS-XXL/2921447485?athbdg=L1600. 
output: {
  \"summary\": \"The Free Assembly Women\'s Patch Pocket Cardigan Sweater is a midweight cardigan sweater that is available in sizes XS-XXL. The reviews for this product are generally positive, with customers praising the sweater\'s soft fabric, comfortable fit, and stylish design. However, some customers have complained that the sweater is too thin and that it does not keep them warm enough. Overall, the Free Assembly Women\'s Patch Pocket Cardigan Sweater is a good option for women who are looking for a stylish and comfortable cardigan sweater.\",
  \"class\": \"Positive\"
}

input: Craft a coherent summary in maximum of 100 words of user reviews for Walmart.com that can be parsed from: https://www.walmart.com/ip/GoPro-HERO9-Black-Waterproof-Action-Camera-Front-LCD-Touch-Rear-Screens-5K-Ultra-HD-Video-20MP-Photos-1080p-Live-Streaming-Webcam-Stabilization/412520823?athcpid=412520823&athpgid=AthenaContentPage_133277&athcgid=null&athznid=ItemCarousel_928590cc-7b51-4ed7-97f9-ccf0b5969f7d_items&athieid=v0&athstid=CS020&athguid=0nJ2IAwDm7RdgGyZ6OEmKBF62aH0cpKNcp73&athancid=null&athena=true&athbdg=L1600
output: {
  \"summary\": \"The GoPro HERO9 Black is a waterproof action camera that is capable of recording 5K video and taking 20MP photos. The camera has a front-facing LCD screen and a rear-facing touch screen, and it can be used for live streaming and as a webcam. The reviews for this product are generally positive, with customers praising the camera\'s image quality, ease of use, and durability. However, some customers have complained about the camera\'s battery life and the price. Overall, the GoPro HERO9 Black is a good option for action camera enthusiasts who are looking for a high-quality camera with a variety of features.\",
  \"class\": \"Positive\"
}
""" +
"""
input: Craft a coherent summary in maximum of 100 words of user reviews for Walmart.com that can be parsed from: \"{}\"

output:
""".format(input_url),
    **parameters)
    try:
        generate_from_web = json.loads(response.text)
        if generate_from_web['class'].lower() == 'positive':
            emoji = "🟢"
        elif generate_from_web['class'].lower() == 'negative':
            emoji = "🔴"
        else:
            emoji = "🟡"
            
    except Exception as ex:
        generate_from_web['summary'] = "Not Much Reviews Available for the conclusion"
        print("Exception:",ex)
    # Markdown(generate_from_web)
    return generate_from_web['summary'], emoji

In [111]:
def generate_summary(input_url):
    generate_from_web = {}
    emoji = "🟡"
    parameters = {
    "max_output_tokens": 1024,
    "temperature": 0.2,
    "top_p": 0.95,
    "top_k": 40
    }
    response = generation_model.predict(
        f"""You are an Review Summarisation Tool, which provides concise summary of users reviews of products purchased from Walmart.com. Analyzing Customer Sentiment and Reviews for Walmart.com.

Craft a coherent summary in maximum of 100 words of user reviews for Walmart.com that can be parsed from: \"{input_url}\"

Focus on extracting key sentiments, prevalent themes and overall satisfaction levels conveyed in the reviews as well as sentiments related to specific categories: price, shipping, delivery, quality, customer service, and features/functionality. Provide both positive and negative feedback where applicable, capturing the general consensus of customer\'s experience.

The coherent summary output should consider below properties wherever applicable:
1. Exclude offensive words from the output.
2. Include overall Sentiment based on these reviews
3. Include Sentiment towards the price point
4. Include Sentiment towards shipping efficiency
5. Include Sentiment towards delivery experience
6. Include Sentiment towards product quality
7. Include Sentiment towards customer service experience
8. Include Sentiment towards product features and functionality
9. Include recommendations to buy the product or not.

Also, classify the coherent summary from the following 3 categories:
    1. Positive
    2. Negative
    3. Neutral

Respond only in consistent json format with summary under \"summary\" key and class under the \"class\" key.""",
    **parameters)
    try:
        generate_from_web = json.loads(response.text)
        if generate_from_web['class'].lower() == 'positive':
            emoji = "🟢"
        elif generate_from_web['class'].lower() == 'negative':
            emoji = "🔴"
        else:
            emoji = "🟡"
            
    except Exception as ex:
        generate_from_web['summary'] = "Not Much Reviews Available for the conclusion"
        print("Exception:",ex)
    # Markdown(generate_from_web)
    return generate_from_web['summary'], emoji

### Fron-End App

In [ ]:
sample_url = [
    ["https://www.walmart.com/ip/Restored-Apple-AirPods-Pro-2-White-In-Ear-Headphones-MQD83AM-A-Refurbished/2379347975?athbdg=L1600"],
    ["https://www.walmart.com/ip/Apple-EarPods-with-Lightning-Connector/54594375?athbdg=L1102"],
    ["https://www.walmart.com/ip/Microsoft-Xbox-Wireless-Controller-Pulse-Red/213327461?athbdg=L1200"],
    ["https://www.walmart.com/ip/Xbox-Starfield-Collectors-Edition-Wireless-Headset-Bluetooth-Connectivity-For-Series-X-S-XBX1-Windows-10-Feat-Auto-mute-voice-isolation-Comfortable-i/1664794666?athbdg=L1600"]
]
css = """
#rowStyle {background-color: #000000}
.gradio-container {background-color: #000000}
.feedback textarea {font-size: 24px !important}
"""
with gr.Blocks(title="Prometheus", theme=gr.themes.Default(primary_hue="red", 
                                                           secondary_hue="red",
                                                           text_size="md",
                                                           spacing_size="md",
                                                           radius_size='md'
                                                           ),
               css=css
              ) as demo:
    with gr.Row(equal_height=True):
        gr.Markdown("""<h1 style="text-align: center;color:white" > 🕵 Prometheus</h1>""")

    with gr.Row(equal_height=True):     
        gr.Markdown("""<h2 style="text-align: center;color:white" >Generate Review Sumary using Vertex AI </h1>""")
    
    with gr.Row(equal_height=True):
        gr.Markdown("""<h3 style="text-align: center;color:white" >Tool Version: 1.0.0 </h3>""")
        
    with gr.Tab("Generate User Summary"):
        with gr.Row():
            review_input = gr.Textbox(lines=1,label="Product URL - User Reviews", placeholder="Enter review of the users",elem_id='rowStyle')
        
        generate_button = gr.Button("Generate Prometheus Summary/Recommendation!!",size="lg")
        
        with gr.Row():
            emoji_output = gr.Textbox(scale=1,label="Prometheus Purchase Indicator", placeholder="🟡",elem_id='rowStyle',text_align="center")
            review_output = gr.Textbox(scale=8,lines=5,label=" 🕵 Promtheus Summary" , placeholder = "Generate to see - What Prometheus has to say??",elem_id='rowStyle')
        
    # gr.Examples(generate_summary,inputs=[review_input],outputs=[review_output],examples=sample_url,cache_examples=True)
    gr.Markdown("""<h3 style="text-align: center;color:white">Sample URLs</h3>""")
    gr.Examples(sample_url,[review_input], elem_id='rowStyle')
    generate_button.click(generate_summary,inputs=[review_input],outputs=[review_output,emoji_output])

## Launch app
demo.queue(concurrency_count=3).launch(server_name="0.0.0.0",share=True) # auth=("admin","prometheus@123")

Running on local URL:  http://0.0.0.0:7862


In [85]:
demo.close()

Closing server running on port: 7862
